In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [3]:
mlflow.get_tracking_uri()

'127.0.0.1'

In [4]:
sentence_length = 106

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS", "True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS", "True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=False


In [19]:
from tooling.config import Experiment, Transformation
#Set Iterations=1 for Split training and testing on datasets. Set dataset to desired dataset split. See tooling.py for split options
base_experiment_config = Experiment(
    name="Base Config 1",
    iterations=1,
    force=False,
    dataset="all",
    lower_case=True,
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Domain_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

In [20]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)

Path("./src/service/models/").mkdir(parents=True, exist_ok=True)

hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

02:18:06 INFO:Hint Label2Id: hint_label2id={'0': 0, 'System_Level': 1, 'Domain_Level': 2, 'Interaction_Level': 3}
02:18:06 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BiLSTM First Stage Model


In [ ]:
from tooling.observability import get_run_id
from tooling.config import BiLSTMConfig, BiLSTM

from copy import deepcopy

bilstm_experiment_config = deepcopy(base_experiment_config)
bilstm_experiment_config.name = "Production"

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)

bilstm_cfg: BiLSTMConfig = OmegaConf.structured(
    BiLSTMConfig(
        bilstm=bilstm_config,
        experiment=bilstm_experiment_config,
        transformation=levels_transformation_config,
    )
)

# bilstm_cfg.experiment.force = True

if run_experiments:
    from experiments.bilstm import bilstm

    bilstm(bilstm_cfg)


bilstm_run_id = get_run_id(bilstm_cfg, pin_commit=pin_commits)

print(bilstm_run_id)
print(mlflow.get_artifact_uri())

bilstm_run = mlflow.get_run(bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bilstm")

## Train SNER First Stage Model


In [ ]:
from tooling.observability import get_run_id
from tooling.config import SNERConfig, SNER

from copy import deepcopy

sner_experiment_config = deepcopy(base_experiment_config)
sner_experiment_config.name = "Production"

sner_config = SNER()

sner_cfg = OmegaConf.structured(
    SNERConfig(
        sner=sner_config,
        experiment=sner_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.sner import sner

    sner(OmegaConf.create(sner_cfg))

sner_run_id = get_run_id(sner_cfg, pin_commit=pin_commits)
print(mlflow.get_artifact_uri())
print(sner_run_id)

sner_run = mlflow.get_run(sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{sner_run.info.artifact_uri}/0_model.ser.gz",
    dst_path=Path("./src/service/models/"),
)
try:
    Path("./src/service/models/sner.ser.gz").unlink()
except FileNotFoundError:
    pass
Path("./src/service/models/0_model.ser.gz").rename(
    "./src/service/models/sner.ser.gz"
)

## Train BERT First Stage Model


In [18]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(base_experiment_config)
bert_1_experiment_config.name = "Production"
bert_1_experiment_config.force = False

bert_1_config = BERT(
    max_len=123,
    number_epochs=11,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=True,
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.0005,
    validation_batch_size=64,
)


bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit=pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")
## Train BERT Second Stage Model for BERT First Stage

11:57:42 INFO:
bert:
  model: bert-large-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 64
  number_epochs: 11
  learning_rate_bert: 3.0e-05
  learning_rate_classifier: 0.0005
  weight_decay: 0.01
  weighted_classes: true
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: paf
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
transformation:
  description: Levels
  type: Reduced
  task: Domain_Level
  goals: null
  domain_data: Domain_Level
  activity: Domain_Level
  stakeholder: Domain_Level
  system_function: Interaction_Level
  interaction: Interaction_Level
  interaction_data: Domain_Level
  workspace: Interaction_Level
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

11:57:42 INFO:New experiment. Running
11:57:42 INFO:Entering 

Entered Iteration 1 now
prolific
anno
prolific


11:58:01 INFO:Created datasets for foldless run
11:58:01 INFO:Starting iteration=1


                                sentence_id sentence_idx   string  \
0      60804326-296b-11ef-88ee-a6448883ebfb            0     this   
1      60804326-296b-11ef-88ee-a6448883ebfb            1      app   
2      60804326-296b-11ef-88ee-a6448883ebfb            2       is   
3      60804326-296b-11ef-88ee-a6448883ebfb            3      for   
4      60804326-296b-11ef-88ee-a6448883ebfb            4  mapping   
...                                     ...          ...      ...   
26602  60cd5842-296b-11ef-a704-a6448883ebfb            2    could   
26603  60cd5842-296b-11ef-a704-a6448883ebfb            3       be   
26604  60cd5842-296b-11ef-a704-a6448883ebfb            4     made   
26605  60cd5842-296b-11ef-a704-a6448883ebfb            5  clearer   
26606  60cd5842-296b-11ef-a704-a6448883ebfb            6        .   

              tore_label  
0                      0  
1           System_Level  
2                      0  
3                      0  
4      Interaction_Level  
...      

Map:   0%|          | 0/1344 [00:00<?, ? examples/s]

...

12:10:36 INFO:Logged iteration result res.precision=0.5539589497846025 res.recall=0.5767043666798917


{'eval_loss': 3.4276018142700195, 'eval_step': 20, 'eval_precision': 0.5539589497846025, 'eval_recall': 0.5767043666798917, 'eval_f1': 0.5651028748348348, 'eval_label_count': 4, 'eval_runtime': 4.6084, 'eval_samples_per_second': 187.702, 'eval_steps_per_second': 3.038, 'epoch': 11.0}
{'train_runtime': 787.9082, 'train_samples_per_second': 18.764, 'train_steps_per_second': 2.345, 'train_loss': 0.11006036039554712, 'epoch': 11.0}


12:11:34 INFO:Logged iteration result res.precision=0.5539589497846025 res.recall=0.5767043666798917
12:11:34 INFO:Logged iteration result res.precision=0.5539589497846025 res.recall=0.5767043666798917
12:11:34 INFO:Finished iteration=1
12:11:34 INFO:Logging model artifact (might take a while)
12:11:35 INFO:Logged experiment result res.mean_precision=0.5539589497846025 res.mean_recall=0.5767043666798917
12:11:37 INFO:Left mlflow context


dcd8f1a95677469bb7ee5ba7c5b2fa8e


MlflowException: The following failures occurred while downloading one or more artifacts from file:///E:/BERT/uvl-tore-classifier-bert/127.0.0.1/279074982670971635/dcd8f1a95677469bb7ee5ba7c5b2fa8e/artifacts:
##### File 0_model #####
[Errno 2] No such file or directory: 'E:\\BERT\\uvl-tore-classifier-bert\\127.0.0.1\\279074982670971635\\dcd8f1a95677469bb7ee5ba7c5b2fa8e\\artifacts\\0_model'

In [35]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bert_2_bert_experiment_config = deepcopy(base_experiment_config)
bert_2_bert_experiment_config.name = "Production"

bert_2_bert_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
    validation_batch_size=64,
)

cfg = DualModelStagedBERTConfig(
    bert=bert_2_bert_config,
    experiment=bert_2_bert_experiment_config,
    transformation=label_transformation_config,
    first_model_bert=bert_1_cfg,
)

bert_2_bert_cfg: DualModelStagedBERTConfig = OmegaConf.structured(cfg)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bert_cfg))

bert_2_bert_run_id = get_run_id(bert_2_bert_cfg, pin_commit=pin_commits)

print(bert_2_bert_run_id)

bert_2_bert_run = mlflow.get_run(bert_2_bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bert_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_2_bert")

05:22:04 INFO:
first_model_bert:
  bert:
    model: bert-large-uncased
    type: BERT
    max_len: 123
    train_batch_size: 8
    validation_batch_size: 64
    number_epochs: 11
    learning_rate_bert: 3.0e-05
    learning_rate_classifier: 0.0005
    weight_decay: 0.01
    weighted_classes: true
  experiment:
    name: Production
    description: ''
    random_state: 125
    folds: 5
    iterations: 1
    average: macro
    dataset: all
    lower_case: true
    force: false
    pin_commit: false
    smote: false
    smote_k_neighbors: 5
    smote_sampling_strategy: not majority
  transformation:
    description: Levels
    type: Reduced
    task: Domain_Level
    goals: null
    domain_data: Domain_Level
    activity: Domain_Level
    stakeholder: Domain_Level
    system_function: Interaction_Level
    interaction: Interaction_Level
    interaction_data: Domain_Level
    workspace: Interaction_Level
    software: System_Level
    internal_action: System_Level
    internal_data: System

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

05:22:22 INFO:Loading Model
05:22:24 INFO:Creating hint column
05:22:39 INFO:Hint column created


Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

...

05:27:38 INFO:Logged iteration result res.precision=0.6208496218507803 res.recall=0.6323361013544841


{'eval_loss': 3.5410361289978027, 'eval_step': 14, 'eval_precision': 0.6208496218507803, 'eval_recall': 0.6323361013544841, 'eval_f1': 0.626540220079135, 'eval_label_count': 10, 'eval_runtime': 1.6119, 'eval_samples_per_second': 385.873, 'eval_steps_per_second': 6.204, 'epoch': 8.0}
{'train_runtime': 302.7632, 'train_samples_per_second': 65.741, 'train_steps_per_second': 8.218, 'train_loss': 2.1282704098814937, 'epoch': 8.0}


05:28:01 INFO:Logged iteration result res.precision=0.6208496218507803 res.recall=0.6323361013544841
05:28:01 INFO:Logged iteration result res.precision=0.6208496218507803 res.recall=0.6323361013544841
05:28:01 INFO:Finished iteration=0
05:28:01 INFO:Logging model artifact (might take a while)
05:28:02 INFO:Breaking early after iteration=0 of 5 folds
05:28:03 INFO:Logged experiment result res.mean_precision=0.6208496218507803 res.mean_recall=0.6323361013544841
05:28:05 INFO:Left mlflow context


1eeb76217ccd4fe29b4d27a19deb209c


WindowsPath('src/service/models/bert_2_bert')

## Train BERT Second Stage Model for SNER First Stage


In [ ]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

bert_2_sner_experiment_config = deepcopy(base_experiment_config)
bert_2_sner_experiment_config.name = "Production"

bert_2_sner_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_sner_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_sner_config,
        experiment=bert_2_sner_experiment_config,
        transformation=label_transformation_config,
        first_model_sner=sner_cfg,
    )
)

if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_sner_cfg))

bert_2_sner_run_id = get_run_id(bert_2_sner_cfg, pin_commit=pin_commits)

print(bert_2_sner_run_id)

bert_2_sner_run = mlflow.get_run(bert_2_sner_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_sner_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_sner"))
except FileNotFoundError:
    pass

Path("./src/service/models/0_model").rename("./src/service/models/bert_2_sner")

## Train BERT Second Stage Model for BILSTM First Stage


In [ ]:
from tooling.observability import get_run_id
from tooling.config import DualModelStagedBERTConfig, StagedBERT

from copy import deepcopy

bilstm_config = BiLSTM(
    batch_size=16,
    learning_rate=0.006,
    number_epochs=5,
    weighted_classes=False,
    sentence_length=106,
)


bert_2_bilstm_experiment_config = deepcopy(base_experiment_config)
bert_2_bilstm_experiment_config.name = "Production"

bert_2_bilstm_config = StagedBERT(
    max_len=123,
    layers=[],
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.16,
    number_epochs=5,
    train_batch_size=8,
    weight_decay=0.1,
    weighted_classes=True,
)

bert_2_bilstm_cfg = OmegaConf.structured(
    DualModelStagedBERTConfig(
        bert=bert_2_bilstm_config,
        experiment=bert_2_bilstm_experiment_config,
        transformation=label_transformation_config,
        first_model_bilstm=bilstm_cfg,
    )
)
bert_2_bilstm_cfg.first_model_bilstm.bilstm.sentence_length = 106


if run_experiments:
    from experiments.dual_model_staged_bert import dual_stage_bert

    dual_stage_bert(OmegaConf.create(bert_2_bilstm_cfg))

bert_2_bilstm_run_id = get_run_id(bert_2_bilstm_cfg, pin_commit=pin_commits)

print(bert_2_bilstm_run_id)

bert_2_bilstm_run = mlflow.get_run(bert_2_bilstm_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_2_bilstm_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_2_bilstm"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename(
    "./src/service/models/bert_2_bilstm"
)

## Train BERT E2E Model


In [11]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(base_experiment_config)
bert_experiment_config.name = "Production"

bert_config = BERT(
    learning_rate_bert=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=11 ,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit=pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

11:17:17 INFO:
bert:
  model: bert-large-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 11
  learning_rate_bert: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: PFTrainKTest
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

11:17:17 WARNING:Experiment was already run
11:17:17 INFO:Entering mlflow context
11:

Entered Iteration 1 now
prolific
anno
komoot


11:17:44 INFO:Created datasets for foldless run


                                sentence_id sentence_idx   string  \
0      baedc5bc-2965-11ef-919a-a6448883ebfb            0     this   
1      baedc5bc-2965-11ef-919a-a6448883ebfb            1      app   
2      baedc5bc-2965-11ef-919a-a6448883ebfb            2       is   
3      baedc5bc-2965-11ef-919a-a6448883ebfb            3      for   
4      baedc5bc-2965-11ef-919a-a6448883ebfb            4  mapping   
...                                     ...          ...      ...   
41032  bb4f7b7d-2965-11ef-ae25-a6448883ebfb           33      and   
41033  bb4f7b7d-2965-11ef-ae25-a6448883ebfb           34     save   
41034  bb4f7b7d-2965-11ef-ae25-a6448883ebfb           35        )   
41035  bb4f7b7d-2965-11ef-ae25-a6448883ebfb           36        >   
41036  bb4f7b7d-2965-11ef-ae25-a6448883ebfb           37    start   

            tore_label  
0                    0  
1         System_Level  
2                    0  
3                    0  
4      System_Function  
...                ..

11:17:44 INFO:Starting iteration=1


...

11:34:32 INFO:Logged iteration result res.precision=0.5896945702388192 res.recall=0.457998609863302


{'eval_loss': 2.88815975189209, 'eval_step': 20, 'eval_precision': 0.5896945702388192, 'eval_recall': 0.457998609863302, 'eval_f1': 0.5155694406390833, 'eval_label_count': 10, 'eval_runtime': 4.9393, 'eval_samples_per_second': 182.414, 'eval_steps_per_second': 5.871, 'epoch': 11.0}
{'train_runtime': 1042.6365, 'train_samples_per_second': 23.305, 'train_steps_per_second': 2.922, 'train_loss': 0.2584537554557964, 'epoch': 11.0}


11:35:28 INFO:Logged iteration result res.precision=0.5896945702388192 res.recall=0.457998609863302
11:35:29 INFO:Logged iteration result res.precision=0.5896945702388192 res.recall=0.457998609863302
11:35:29 INFO:Finished iteration=1
11:35:29 INFO:Logging model artifact (might take a while)
11:35:32 INFO:Logged experiment result res.mean_precision=0.5896945702388192 res.mean_recall=0.457998609863302
11:35:33 INFO:Left mlflow context


7cfcddb65af24a4595672177cf4f600d


MlflowException: The following failures occurred while downloading one or more artifacts from file:///E:/BERT/uvl-tore-classifier-bert/127.0.0.1/279074982670971635/7cfcddb65af24a4595672177cf4f600d/artifacts:
##### File 0_model #####
[Errno 2] No such file or directory: 'E:\\BERT\\uvl-tore-classifier-bert\\127.0.0.1\\279074982670971635\\7cfcddb65af24a4595672177cf4f600d\\artifacts\\0_model'

In [ ]:
##Train RoBERTa E2E Model


In [14]:
from tooling.observability import get_run_id
from tooling.config import RoBERTaConfig, RoBERTa

from copy import deepcopy

roberta_experiment_config = deepcopy(base_experiment_config)
roberta_experiment_config.name = "Production"

roberta_config = RoBERTa(
    learning_rate_roberta=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=11,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

roberta_cfg = OmegaConf.structured(
    RoBERTaConfig(
        roberta=roberta_config,
        experiment=roberta_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.roberta import roberta

    roberta(OmegaConf.create(roberta_cfg))

roberta_run_id = get_run_id(roberta_cfg, pin_commit=pin_commits)

print(roberta_run_id)

run = mlflow.get_run(roberta_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/roberta"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/roberta")


01:44:16 INFO:
roberta:
  model: Jean-Baptiste/roberta-large-ner-english
  type: RoBERTa
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 11
  learning_rate_roberta: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: Production
  description: ''
  random_state: 125
  folds: 5
  iterations: 1
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: false
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

01:44:16 INFO:New experiment. Running
01:44:16 INFO:Entering mlf

Map:   0%|          | 0/2488 [00:00<?, ? examples/s]

Map:   0%|          | 0/622 [00:00<?, ? examples/s]

E:\BERT\uvl-tore-classifier-bert\venv310\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
01:45:21 INFO:Logged iteration result res.precision=0.5793384603014975 res.recall=0.5294931132606071


{'eval_loss': 0.606141209602356, 'eval_step': 0, 'eval_precision': 0.5793384603014975, 'eval_recall': 0.5294931132606071, 'eval_f1': 0.5532954369096804, 'eval_label_count': 10, 'eval_runtime': 3.4457, 'eval_samples_per_second': 180.515, 'eval_steps_per_second': 5.804, 'epoch': 1.0}
{'loss': 1.2792, 'grad_norm': 15.624954223632812, 'learning_rate': 5.123063431745104e-05, 'epoch': 1.607717041800643}


...

02:01:44 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085


{'eval_loss': 0.9886234998703003, 'eval_step': 20, 'eval_precision': 0.6904512276980231, 'eval_recall': 0.6926464362913085, 'eval_f1': 0.6915470899120589, 'eval_label_count': 10, 'eval_runtime': 3.5215, 'eval_samples_per_second': 176.631, 'eval_steps_per_second': 5.679, 'epoch': 11.0}
{'train_runtime': 1083.2292, 'train_samples_per_second': 25.265, 'train_steps_per_second': 3.158, 'train_loss': 0.45614679554546483, 'epoch': 11.0}


02:02:35 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085
02:02:35 INFO:Logged iteration result res.precision=0.6904512276980231 res.recall=0.6926464362913085
02:02:35 INFO:Finished iteration=0
02:02:35 INFO:Logging model artifact (might take a while)
02:02:38 INFO:Breaking early after iteration=0 of 5 folds
02:02:38 INFO:Logged experiment result res.mean_precision=0.6904512276980231 res.mean_recall=0.6926464362913085
02:02:39 INFO:Left mlflow context


73c3721d2c3e41e6b0539148725132df


WindowsPath('src/service/models/roberta')